In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm

import requests
from datetime import datetime
import json
import pickle

## 데이터 불러오기

In [2]:
df = pd.read_csv("data/request_reviews.csv")
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,NaN,,
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240


In [3]:
df.shape

(12528, 20)

In [4]:
df[df.keyword.isna()]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,NaN,['338342608'],338342608,338342608
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,203668.426416,443485.604196,기타,상수도전용,역삼동,NaN,NaN,['10960059'],10960059,10960059
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,205130.591679,445590.096838,NaN,NaN,삼성동,NaN,NaN,['16877892'],16877892,16877892
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,204279.144297,447023.007955,NaN,NaN,청담동,NaN,NaN,['19153490'],19153490,19153490
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,203566.263883,443554.896146,NaN,NaN,역삼동,NaN,NaN,['14600986'],14600986,14600986
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,202951.131852,444272.491202,NaN,NaN,역삼동,NaN,NaN,NaN,,
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,202024.615630,444994.659485,NaN,NaN,논현동,NaN,NaN,['522500805'],522500805,522500805
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,203049.750409,447323.452649,NaN,NaN,신사동,NaN,NaN,['1495236287'],1495236287,1495236287
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,204171.424581,446636.433530,NaN,NaN,청담동,NaN,NaN,['118575691'],118575691,118575691
9354,9354,3220000-101-2021-00858,20210708,영업/정상,159.25,135915,서울특별시 강남구 역삼동 676 센터필드,서울 강남구 테헤란로 231,(유)한국집 센터필드 역삼점,한식,203599.062772,444556.062267,NaN,NaN,역삼동,NaN,NaN,['1872560550'],1872560550,1872560550


In [5]:
# df["url"] = df["id_mod2"].apply(lambda x: np.nan if x==" " else x)
df["url"] = df["id_mod2"].fillna(" ")
df["url"] = df["url"].apply(lambda x: "https://place.map.kakao.com/" + x if x!=" " else x)

In [6]:
df["keyword2"] = df["keyword"].fillna(" ")

# 업소 이름 뒤에 위치한 동 이름 붙임
def plus_dong(txt):
    if txt["dong"][:-1] not in txt["keyword2"]:
        txt["name"] = txt["keyword2"] + " " + txt["dong"][:-1]
    else:
        txt["name"] = txt["keyword2"]
    return txt

In [7]:
# plus_dong 함수 적용
df = df.apply(plus_dong, axis=1)

## request 적용 -> 실패

In [8]:
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com/10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com/727661240,로비라운지,로비라운지 삼성


In [9]:
df[df.id_mod2 != " "].shape


(6465, 23)

In [10]:
df["id_mod2"][:3]

0             
1     10825881
2    727661240
Name: id_mod2, dtype: object

In [11]:
request_id = 14569639
url = f'https://place.map.kakao.com/main/v/{request_id}'
response = requests.get(url)
comments = []
if response.status_code == 200:
    comment_data = json.loads(response.content)['comment']
    comment_count = int(comment_data['kamapComntcnt'])
    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
    for c in comment_data['list']:
        comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
        request_comment_id = c['commentid']
    print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
else:
    print('error')

총 댓글 개수: 169개, 평점: 4.11점


In [12]:
while len(comments) < comment_count:
    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
    response = requests.get(url)
    if response.status_code == 200:
        comment_data = json.loads(response.content)['comment']
        for c in comment_data['list']:
            try:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
            except:
                comments.append({'점수': c['point'], '댓글': '', '작성일': c['date']})
            request_comment_id = c['commentid']
    else:
        print('error')

In [13]:
pd.DataFrame(comments)

,점수,댓글,작성일
0,5,,2024.04.22.
1,2,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...,2024.04.14.
2,5,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...,2024.04.11.
3,5,수육이랑 곰탕먹었어요\n곰탕은 주문하자마자 빨리 나왔어요\n고기도 많이 들어가있고\...,2024.04.08.
4,5,세종대왕의 이름 (이도 李裪)에 걸맞는 \n진한 맛의 곰탕입니다\n마침 제옆에 외국...,2024.04.02.
...,...,...,...
164,4,곰탕,2016.09.29.
165,4,소수육46. 면+죽 추가10.,2016.09.09.
166,2,공장형.\n밥대신 국수.\n잘 모르겠다.,2016.08.03.
167,4,,2016.07.10.


In [14]:
comment_data

{'kamapComntcnt': 169,
 'scoresum': 695,
 'scorecnt': 169,
 'list': [{'commentid': '842665',
   'contents': '(수육+장칼국수+죽) 3명에서 이렇게 먹으니 딱좋네.\n복잡한 역삼역에 주차공간도 나름 넉넉 (10대정도?)\n다음에는 곰탕을 먹어 봐야겠다.',
   'point': 5,
   'username': '자유로',
   'profile': 'http://t1.daumcdn.net/local/kakaomapPhoto/profile/381d738b852dead3074f44ea9effa8bf24a6f1bb?original',
   'profileStatus': 'S',
   'photoCnt': 1,
   'likeCnt': 0,
   'thumbnail': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
   'kakaoMapUserId': '267318248',
   'photoList': [{'url': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
     'near': False}],
   'ownerReply': {},
   'userCommentCount': 72,
   'userCommentAverageScore': 4.2,
   'myStorePick': False,
   'level': {'nowLevel': 28, 'badge': '02'},
   'date': '2016.03.15.',
   'isMy': False,
   'isBlock': False,
   'isEditable': False,
   'isMyLike': False}],
 'strengthCounts': [{'id': 5, 'nam

In [15]:
df["id_mod2"].shape[0]/3, df["id_mod2"].shape[0]/3*2, df["id_mod2"].shape[0]/3*3

(4176.0, 8352.0, 12528.0)

In [26]:
comments = []

# for i, id in enumerate(df["id_mod2"][8352:], 8352):
for i, id in enumerate(df["id_mod2"][8352:8355], 8352):
    
    if id==" ":
        continue
    else:
        try:
            time.sleep(1)
            request_id = id
            url = f'https://place.map.kakao.com/main/v/{request_id}'
            response = requests.get(url)
            # print(url)

            try:

                if response.status_code == 200:
                    comment_num = 3
                    comment_data = json.loads(response.content)['comment']
                    comment_count = int(comment_data['kamapComntcnt'])
                    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
                    for c in comment_data['list']:
                        # comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                'review': c['contents'], 'date': c['date'],
                                                'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                        request_comment_id = c['commentid']
                    
                    while comment_num < comment_count:
                        url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
                        response = requests.get(url)
                        if response.status_code == 200:
                            comment_data = json.loads(response.content)['comment']
                            for c in comment_data['list']:
                                comment_num += 1
                                try:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': c['contents'], 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                except:
                                    comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                                    'review': "", 'date': c['date'],
                                                    'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                                    "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                                request_comment_id = c['commentid']
                            # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
                else:
                    print('response')
                    # pass
            except:
                print('comments error')
        
        except:
            print("url_error")
            
        

In [27]:
pd.DataFrame(comments)

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,제주흑돼지마을,맛집탐방아티스트,친절하고 조용해서 작업하기 좋음,2021.11.20.,5,4.3,413,NaN
1,제주흑돼지마을,재윤.,이정도면 친절,2021.08.27.,5,3.7,18,NaN
2,제주흑돼지마을,시러시라,일하는 여자알바 불친절 ~,2021.02.25.,1,3.4,5,NaN
3,부산아지매국밥 삼성중앙,.,,2022.12.28.,3,3.5,245,NaN
4,부산아지매국밥 삼성중앙,Jiyoon,가끔 정갈한 한식이 먹고 싶을 때 들리는 레스토랑입니다.\n이번에 1층 옷 매장 갔...,2022.11.18.,5,5.0,1,NaN
5,부산아지매국밥 삼성중앙,직설맨,,2022.09.01.,1,3.0,239,NaN
6,부산아지매국밥 삼성중앙,navi,분위기는 좋게 만들었는데 뭔가.. 빠진 느낌 꼭 리조트 조식같은 분위기의 음식점 응...,2021.11.27.,1,3.8,93,NaN
7,부산아지매국밥 삼성중앙,tx,,2021.10.16.,3,3.9,116,NaN
8,부산아지매국밥 삼성중앙,lee,"남해 사우스케이프가 좋았어서 갔지만, 깔끔하지만\n냉랭함과 불성실, 불쾌함.",2021.07.05.,1,3.2,19,NaN
9,부산아지매국밥 삼성중앙,자몽,,2021.06.02.,5,3.7,18,NaN


In [22]:
reviews_8352 = pd.DataFrame(comments)
reviews_8352

,name,user,review,date,rate,avg_rate,review_cnt,is_model
0,제주흑돼지마을,맛집탐방아티스트,친절하고 조용해서 작업하기 좋음,2021.11.20.,5,4.3,413,NaN
1,제주흑돼지마을,재윤.,이정도면 친절,2021.08.27.,5,3.7,18,NaN
2,제주흑돼지마을,시러시라,일하는 여자알바 불친절 ~,2021.02.25.,1,3.4,5,NaN
3,부산아지매국밥 삼성중앙,.,,2022.12.28.,3,3.5,245,NaN
4,부산아지매국밥 삼성중앙,Jiyoon,가끔 정갈한 한식이 먹고 싶을 때 들리는 레스토랑입니다.\n이번에 1층 옷 매장 갔...,2022.11.18.,5,5.0,1,NaN
...,...,...,...,...,...,...,...,...
3297,백억하누,곰곰,★★★★★필독★★★★필독★★★★★★\n평점 조작하는 곳입니다. 장난치는것도 아니고 ...,2023.12.06.,1,1.0,1,NaN
3298,백억하누,TAKANO YUMI (병호&병학),,2023.11.27.,5,5.0,7,NaN
3299,월출산,대방어,,2024.04.23.,1,3.5,489,NaN
3300,월출산,김지성,"mild seaweed soup, a bit hard and cold rice, o...",2024.04.11.,4,4.3,7,NaN


In [25]:
reviews_8352.name.nunique()

1184

In [29]:
df[df["keyword"]=="월출산"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
5878,5878,3220000-101-2016-00412,20160520,영업/정상,145.00,135881,서울특별시 강남구 삼성동 162-15,서울 강남구 봉은사로112길 6,월출산,한식,...,NaN,삼성동,모범식당지정,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성
12352,12352,3220000-101-2024-00311,2024-03-14,영업/정상,303.00,135-881,서울특별시 강남구 삼성동 165-11 상원빌딩,서울 강남구 봉은사로112길 22,월출산,한식,...,NaN,삼성동,NaN,월출산,NaN,,107246805,https://place.map.kakao.com/107246805,월출산,월출산 삼성


In [34]:
df[df["keyword"].fillna("").str.contains("베러디")]


,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
8359,8359,3220000-101-2020-00622,20200611,영업/정상,61.30,135811,서울특별시 강남구 논현동 12번지,서울 강남구 논현로149길 65,베러디(betterd),기타,...,NaN,논현동,NaN,베러디,NaN,,,,베러디,베러디 논현


In [36]:
df.loc[8354]

Unnamed: 0                                          8354
MGTNO                             3220000-101-2020-00613
APVPERMYMD                                      20200609
TRDSTATENM                                         영업/정상
SITEAREA                                           62.49
SITEPOSTNO                                        135877
SITEWHLADDR                서울특별시 강남구 삼성동 144-14번지 채널리저브 
RDNWHLADDR                                서울 강남구 삼성로 517
BPLCNM                                     부산아지매국밥 삼성중앙점
UPTAENM                                               한식
X                                           204828.18035
Y                                          445089.672895
TRDPJUBNSENM                                         NaN
WTRSPLYFACILSENM                                     NaN
dong                                                 삼성동
exemplary_diner                                      NaN
keyword                                     부산아지매국밥 삼성중앙
id                             

In [38]:
df[df["keyword"].fillna("").str.contains("사우스")]


,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
6509,6509,3220000-101-2017-00857,20170905,영업/정상,145.52,135895,서울특별시 강남구 신사동 627-12,서울 강남구 언주로 823,사우스(SOUTH),식육(숯불구이),...,NaN,신사동,NaN,사우스,['312947795'],312947795,312947795,https://place.map.kakao.com/312947795,사우스,사우스 신사
7712,7712,3220000-101-2019-00880,20190809,영업/정상,803.46,135893,서울특별시 강남구 신사동 603,서울 강남구 논현로 854,안다즈 서울강남-미들&사우스하우스,경양식,...,NaN,신사동,NaN,안다즈 서울강남-미들&사우스하우스,NaN,,,,안다즈 서울강남-미들&사우스하우스,안다즈 서울강남-미들&사우스하우스 신사
8361,8361,3220000-101-2020-00625,2020-06-12,영업/정상,327.18,135-895,서울특별시 강남구 신사동 631-34 제성빌딩,서울 강남구 도산대로45길 17,메종 사우스케이프,기타,...,NaN,신사동,NaN,메종 사우스케이프,['1687176590'],1687176590,1687176590,https://place.map.kakao.com/1687176590,메종 사우스케이프,메종 사우스케이프 신사
10086,10086,3220000-101-2022-00495,20220428,영업/정상,293.66,135895,서울특별시 강남구 신사동 627-12 ROY빌딩,서울 강남구 언주로 823,사우스(SOUTH),한식,...,NaN,신사동,NaN,사우스,NaN,,,,사우스,사우스 신사


In [52]:
df.loc[200:210]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
200,200,3220000-101-1988-16620,19880326,영업/정상,158.67,135821,서울특별시 강남구 논현동 114-14번지,서울 강남구 언주로136길 10,퍼남사이곤,한식,...,상수도전용,논현동,NaN,퍼남사이곤,['166678875'],166678875,166678875,https://place.map.kakao.com/166678875,퍼남사이곤,퍼남사이곤 논현
201,201,3220000-101-1988-16788,19880909,영업/정상,26.51,135821,서울특별시 강남구 논현동 114-11번지,서울 강남구 언주로134길 9,신의주 부대찌게,한식,...,상수도전용,논현동,NaN,신의주 부대찌게,['18202075'],18202075,18202075,https://place.map.kakao.com/18202075,신의주 부대찌게,신의주 부대찌게 논현
202,202,3220000-101-1988-16795,19881107,영업/정상,186.36,135816,"서울특별시 강남구 논현동 63-22번지 지상1,2층",서울 강남구 언주로147길 4,파이굽는집,한식,...,상수도전용,논현동,NaN,파이굽는집,['27490966'],27490966,27490966,https://place.map.kakao.com/27490966,파이굽는집,파이굽는집 논현
203,203,3220000-101-1988-16800,19881130,영업/정상,195.71,135818,서울특별시 강남구 논현동 83-16번지,서울 강남구 언주로133길 14,베니스,한식,...,상수도전용,논현동,NaN,베니스,NaN,,,,베니스,베니스 논현
204,204,3220000-101-1988-16801,19881201,영업/정상,105.80,135821,서울특별시 강남구 논현동 114-25,서울 강남구 언주로134길 13,해솔X므로,한식,...,NaN,논현동,NaN,해솔X므로,['1037166274'],1037166274,1037166274,https://place.map.kakao.com/1037166274,해솔X므로,해솔X므로 논현
205,205,3220000-101-1988-17156,19880429,영업/정상,198.90,135883,"서울특별시 강남구 수서동 490-5번지 지하1층,지상1층",서울 강남구 광평로 222,정선,한식,...,상수도전용,수서동,NaN,정선,NaN,,,,정선,정선 수서
206,206,3220000-101-1988-17158,19880916,영업/정상,29.11,135516,서울특별시 강남구 일원동 683-17번지,서울 강남구 일원로5길 3,종로빈대떡,한식,...,상수도전용,일원동,NaN,종로빈대떡,NaN,,,,종로빈대떡,종로빈대떡 일원
207,207,3220000-101-1988-17468,19880804,영업/정상,41.41,135887,서울특별시 강남구 신사동 512-10번지,서울 강남구 압구정로2길 62,신사소곱창,한식,...,상수도전용,신사동,NaN,신사소곱창,['13571132'],13571132,13571132,https://place.map.kakao.com/13571132,신사소곱창,신사소곱창
208,208,3220000-101-1988-18479,19880802,영업/정상,99.00,135915,서울특별시 강남구 역삼동 670-22번지,서울 강남구 논현로94길 21,양궈푸 마라탕,한식,...,상수도전용,역삼동,NaN,양궈푸 마라탕,['1221274774'],1221274774,1221274774,https://place.map.kakao.com/1221274774,양궈푸 마라탕,양궈푸 마라탕 역삼
209,209,3220000-101-1988-19171,1988-10-14,영업/정상,35.52,135-891,서울특별시 강남구 신사동 564-14,서울 강남구 도산대로25길 20,태능숯불갈비,한식,...,NaN,신사동,NaN,태능숯불갈비,NaN,,,,태능숯불갈비,태능숯불갈비 신사


In [58]:
df[210:220]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
210,210,3220000-101-1988-20393,19880829,영업/정상,695.98,135090,서울특별시 강남구 삼성동 159-8 인터콘티넨탈 호텔 서울1층,서울 강남구 테헤란로 521,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['13559984'],13559984,13559984,https://place.map.kakao.com/13559984,로비라운지,로비라운지 삼성
211,211,3220000-101-1989-07606,19890120,영업/정상,606.84,135935,"서울특별시 강남구 역삼동 827-58번지 에도긴빌딩2,3층",서울 강남구 테헤란로8길 30,주식회사 에도긴,일식,...,상수도전용,역삼동,모범식당지정,주식회사 에도긴,NaN,,21332162,https://place.map.kakao.com/21332162,주식회사 에도긴,주식회사 에도긴 역삼
212,212,3220000-101-1989-07862,19890918,영업/정상,668.74,135832,서울특별시 강남구 논현동 248-7,서울 강남구 언주로 640,만요,일식,...,상수도전용,논현동,NaN,만요,NaN,,,,만요,만요 논현
213,213,3220000-101-1989-07863,19891204,영업/정상,83.04,135951,서울특별시 강남구 청담동 40-31번지,서울 강남구 선릉로132길 3-3,풍년돌구이,일식,...,상수도전용,청담동,NaN,풍년돌구이,NaN,,,,풍년돌구이,풍년돌구이 청담
214,214,3220000-101-1989-10816,1989-01-19,영업/정상,99.17,135-931,서울특별시 강남구 역삼동 809-6,서울 강남구 봉은사로2길 16,목구멍 신논현점,일식,...,NaN,역삼동,NaN,목구멍 신논현,['1116539400'],1116539400,1116539400,https://place.map.kakao.com/1116539400,목구멍 신논현,목구멍 신논현 역삼
215,215,3220000-101-1989-15222,19890327,영업/정상,55.17,135962,서울특별시 강남구 개포동 1218-27번지,서울 강남구 개포로22길 13,노숙자아구샤브샤브,한식,...,상수도전용,개포동,NaN,노숙자아구샤브샤브,['20745398'],20745398,20745398,https://place.map.kakao.com/20745398,노숙자아구샤브샤브,노숙자아구샤브샤브 개포
216,216,3220000-101-1989-15513,19890329,영업/정상,103.49,135501,서울특별시 강남구 대치동 990-0번지,서울 강남구 도곡로 516,깐부치킨,한식,...,상수도전용,대치동,NaN,깐부치킨,NaN,,,,깐부치킨,깐부치킨 대치
217,217,3220000-101-1989-15515,19890502,영업/정상,59.40,135850,서울특별시 강남구 대치동 974번지 대치현대아파트,서울 강남구 삼성로64길 5,영동식당,한식,...,상수도전용,대치동,NaN,영동식당,NaN,,,,영동식당,영동식당 대치
218,218,3220000-101-1989-15812,19890213,영업/정상,181.80,135882,서울특별시 강남구 삼성동 170-3번지,서울 강남구 테헤란로107길 12,목포낙지횟집,한식,...,상수도전용,삼성동,NaN,목포낙지횟집,['16765211'],16765211,16765211,https://place.map.kakao.com/16765211,목포낙지횟집,목포낙지횟집 삼성
219,219,3220000-101-1989-16238,19890912,영업/정상,42.50,135930,서울특별시 강남구 역삼동 798-29번지,서울 강남구 논현로63길 19,주주형제 생갈비 김치찌개,한식,...,상수도전용,역삼동,NaN,주주형제 생갈비 김치찌개,['1278555240'],1278555240,1278555240,https://place.map.kakao.com/1278555240,주주형제 생갈비 김치찌개,주주형제 생갈비 김치찌개 역삼


In [59]:
df.loc[220:250]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
220,220,3220000-101-1989-16240,1989-10-06,영업/정상,62.70,135-514,서울특별시 강남구 역삼동 732-17,서울 강남구 역삼로37길 3,옛날그맛명동칼국수,한식,...,NaN,역삼동,NaN,옛날그맛명동칼국수,['16819689'],16819689,16819689,https://place.map.kakao.com/16819689,옛날그맛명동칼국수,옛날그맛명동칼국수 역삼
221,221,3220000-101-1989-16245,19891026,영업/정상,88.28,135936,서울특별시 강남구 역삼동 831-36번지,서울 강남구 역삼로3길 15,유가네전사랑,한식,...,상수도전용,역삼동,NaN,유가네전사랑,['1967128130'],1967128130,1967128130,https://place.map.kakao.com/1967128130,유가네전사랑,유가네전사랑 역삼
222,222,3220000-101-1989-16252,19891206,영업/정상,36.00,135930,서울특별시 강남구 역삼동 797-4번지,서울 강남구 논현로 333,황소양곱창,한식,...,상수도전용,역삼동,NaN,황소양곱창,['16647845'],16647845,16647845,https://place.map.kakao.com/16647845,황소양곱창,황소양곱창 역삼
223,223,3220000-101-1989-16264,19890708,영업/정상,80.00,135920,서울특별시 강남구 역삼동 721-11,서울 강남구 언주로81길 6,범골,한식,...,NaN,역삼동,NaN,범골,['63833790'],63833790,63833790,https://place.map.kakao.com/63833790,범골,범골 역삼
224,224,3220000-101-1989-16285,19890323,영업/정상,82.00,135907,서울특별시 강남구 역삼동 604-7,서울 강남구 봉은사로 150,삼정사우나식당,한식,...,상수도전용,역삼동,NaN,삼정사우나식당,NaN,,,,삼정사우나식당,삼정사우나식당 역삼
225,225,3220000-101-1989-16516,19890821,영업/정상,100.00,135953,서울특별시 강남구 청담동 76-11번지,서울 강남구 학동로81길 7,진성식당,한식,...,상수도전용,청담동,모범식당지정,진성식당,['16418709'],16418709,16418709,https://place.map.kakao.com/16418709,진성식당,진성식당 청담
226,226,3220000-101-1989-16780,1989-09-07,영업/정상,149.70,135-821,서울특별시 강남구 논현동 115-3,서울 강남구 언주로134길 8,청담 돈고집,중국식,...,NaN,논현동,NaN,청담 돈고집,['1992125752'],1992125752,1992125752,https://place.map.kakao.com/1992125752,청담 돈고집,청담 돈고집 논현
227,227,3220000-101-1989-16782,1989-09-23,영업/정상,79.52,135-827,서울특별시 강남구 논현동 198-12,서울 강남구 봉은사로5길 9,유쾌한잔 본점,한식,...,NaN,논현동,NaN,유쾌한잔 본,NaN,,,,유쾌한잔 본,유쾌한잔 본 논현
228,228,3220000-101-1989-17126,19890314,영업/정상,71.40,135943,서울특별시 강남구 일원동 619번지,서울 강남구 개포로 626,왕소금구이,한식,...,상수도전용,일원동,NaN,왕소금구이,NaN,,,,왕소금구이,왕소금구이 일원
229,229,3220000-101-1989-17318,1989-02-24,영업/정상,92.00,135-816,서울특별시 강남구 논현동 65-2,서울 강남구 논현로150길 16,전주식당,분식,...,NaN,논현동,NaN,전주식당,['8059543'],8059543,8059543,https://place.map.kakao.com/8059543,전주식당,전주식당 논현


In [61]:
df[df.keyword.isna()]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,...,상수도전용,도곡동,NaN,NaN,['338342608'],338342608,338342608,https://place.map.kakao.com/338342608,,도곡
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,...,상수도전용,역삼동,NaN,NaN,['10960059'],10960059,10960059,https://place.map.kakao.com/10960059,,역삼
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,...,NaN,삼성동,NaN,NaN,['16877892'],16877892,16877892,https://place.map.kakao.com/16877892,,삼성
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,...,NaN,청담동,NaN,NaN,['19153490'],19153490,19153490,https://place.map.kakao.com/19153490,,청담
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,...,NaN,역삼동,NaN,NaN,['14600986'],14600986,14600986,https://place.map.kakao.com/14600986,,역삼
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,...,NaN,역삼동,NaN,NaN,NaN,,,,,역삼
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,...,NaN,논현동,NaN,NaN,['522500805'],522500805,522500805,https://place.map.kakao.com/522500805,,논현
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,...,NaN,신사동,NaN,NaN,['1495236287'],1495236287,1495236287,https://place.map.kakao.com/1495236287,,신사
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,...,NaN,청담동,NaN,NaN,['118575691'],118575691,118575691,https://place.map.kakao.com/118575691,,청담
9354,9354,3220000-101-2021-00858,20210708,영업/정상,159.25,135915,서울특별시 강남구 역삼동 676 센터필드,서울 강남구 테헤란로 231,(유)한국집 센터필드 역삼점,한식,...,NaN,역삼동,NaN,NaN,['1872560550'],1872560550,1872560550,https://place.map.kakao.com/1872560550,,역삼


In [56]:
df.loc[290:300]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
290,290,3220000-101-1991-06000,19910524,영업/정상,29.76,135509,서울특별시 강남구 삼성동 116-0번지,서울 강남구 봉은사로 466,동호치킨,분식,...,상수도전용,삼성동,NaN,동호치킨,['17640526'],17640526,17640526,https://place.map.kakao.com/17640526,동호치킨,동호치킨 삼성
291,291,3220000-101-1991-06005,19911216,영업/정상,55.44,135509,서울특별시 강남구 삼성동 116-34번지,서울 강남구 봉은사로74길 18,초가집,한식,...,상수도전용,삼성동,NaN,초가집,['16708939'],16708939,16708939,https://place.map.kakao.com/16708939,초가집,초가집 삼성
292,292,3220000-101-1991-06012,19910415,영업/정상,31.59,135881,서울특별시 강남구 삼성동 165-8번지,서울 강남구 봉은사로108길 19,허수아비돈까스,한식,...,상수도전용,삼성동,NaN,허수아비돈까스,NaN,,,,허수아비돈까스,허수아비돈까스 삼성
293,293,3220000-101-1991-06016,19910515,영업/정상,44.82,135881,서울특별시 강남구 삼성동 165-8번지,서울 강남구 봉은사로108길 19,텐텐,분식,...,상수도전용,삼성동,NaN,텐텐,NaN,,,,텐텐,텐텐 삼성
294,294,3220000-101-1991-06178,19910702,영업/정상,20.06,135514,서울특별시 강남구 역삼동 732-18번지,서울 강남구 역삼로 247,향토통닭구이,한식,...,상수도전용,역삼동,NaN,향토통닭구이,['18865294'],18865294,18865294,https://place.map.kakao.com/18865294,향토통닭구이,향토통닭구이 역삼
295,295,3220000-101-1991-06498,19910101,영업/정상,21.07,135822,서울특별시 강남구 논현동 122-25번지 지상1층 102호,서울 강남구 강남대로128길 27,페리카나,분식,...,상수도전용,논현동,NaN,페리카나,['8501640'],8501640,8501640,https://place.map.kakao.com/8501640,페리카나,페리카나 논현
296,296,3220000-101-1991-06663,19910723,영업/정상,139.64,135897,서울특별시 강남구 신사동 664-0,서울 강남구 압구정로 344,영양센타압구정점,분식,...,상수도전용,신사동,NaN,영양센타압구정,['23203958'],23203958,23203958,https://place.map.kakao.com/23203958,영양센타압구정,영양센타압구정 신사
297,297,3220000-101-1991-06716,19910426,영업/정상,105.40,135888,서울특별시 강남구 신사동 522-0번지 신사아이빌 지하1층 B101호,서울 강남구 압구정로2길 15,김북순큰남비집,분식,...,상수도전용,신사동,NaN,김북순큰남비집,['16679761'],16679761,16679761,https://place.map.kakao.com/16679761,김북순큰남비집,김북순큰남비집 신사
298,298,3220000-101-1991-06821,19910320,영업/정상,25.20,135945,서울특별시 강남구 일원동 663-8번지 지상1층,서울 강남구 일원로5길 51,밤푸스치킨,분식,...,상수도전용,일원동,NaN,밤푸스치킨,['12890589'],12890589,12890589,https://place.map.kakao.com/12890589,밤푸스치킨,밤푸스치킨 일원
299,299,3220000-101-1991-07549,19911001,영업/정상,142.24,135875,서울특별시 강남구 삼성동 140-29번지,서울 강남구 선릉로94길 9,(주)김돈이,한식,...,상수도전용,삼성동,NaN,김돈이,['10800261'],10800261,10800261,https://place.map.kakao.com/10800261,김돈이,김돈이 삼성


In [54]:
df.loc[300:310]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
300,300,3220000-101-1991-07598,19911012,영업/정상,227.23,135933,서울특별시 강남구 역삼동 823-24번지,서울 강남구 테헤란로14길 6,조수사,일식,...,상수도전용,역삼동,모범식당지정,조수사,['978587274'],978587274,10800261,https://place.map.kakao.com/10800261,조수사,조수사 역삼
301,301,3220000-101-1991-07749,1991-11-11,영업/정상,156.92,135-887,서울특별시 강남구 신사동 514-10 경서빌딩,서울 강남구 도산대로 127,초장집 신사가로수길점,한식,...,NaN,신사동,NaN,초장집 신사가로수길,['2106188174'],2106188174,2106188174,https://place.map.kakao.com/2106188174,초장집 신사가로수길,초장집 신사가로수길
302,302,3220000-101-1991-07802,19910810,영업/정상,184.04,135995,서울특별시 강남구 논현동 70-0번지,서울 강남구 언주로147길 9,(주)예,일식,...,상수도전용,논현동,NaN,예,['7929136'],7929136,7929136,https://place.map.kakao.com/7929136,예,예 논현
303,303,3220000-101-1991-07853,1991-06-24,영업/정상,107.06,135-816,서울특별시 강남구 논현동 64-0,서울 강남구 도산대로34길 11,삼호짱뚱이,한식,...,NaN,논현동,NaN,삼호짱뚱이,['1421668312'],1421668312,1421668312,https://place.map.kakao.com/1421668312,삼호짱뚱이,삼호짱뚱이 논현
304,304,3220000-101-1991-10322,19911101,영업/정상,86.58,135998,서울특별시 강남구 대치동 935-3,서울 강남구 선릉로64길 26,나따오비까(대치점),경양식,...,상수도전용,대치동,NaN,나따오비까,['22067875'],22067875,22067875,https://place.map.kakao.com/22067875,나따오비까,나따오비까 대치
305,305,3220000-101-1991-12071,19910727,영업/정상,127.79,135894,서울특별시 강남구 신사동 613-15번지,서울 강남구 논현로176길 28,뉴델리,경양식,...,상수도전용,신사동,NaN,뉴델리,['17532348'],17532348,17532348,https://place.map.kakao.com/17532348,뉴델리,뉴델리 신사
306,306,3220000-101-1991-12387,19910802,영업/정상,28.60,135961,서울특별시 강남구 개포동 1184-1번지,서울 강남구 논현로10길 26,피자노베,분식,...,상수도전용,개포동,NaN,피자노베,NaN,,,,피자노베,피자노베 개포
307,307,3220000-101-1991-12443,19910801,영업/정상,248.24,135882,서울특별시 강남구 삼성동 169-6번지,서울 강남구 테헤란로103길 10,동강,중국식,...,상수도전용,삼성동,NaN,동강,NaN,,,,동강,동강 삼성
308,308,3220000-101-1991-12806,19911126,영업/정상,47.88,135859,서울특별시 강남구 도곡동 942-8번지,서울 강남구 강남대로58길 21,두메,중국식,...,상수도전용,도곡동,NaN,두메,['604735311'],604735311,604735311,https://place.map.kakao.com/604735311,두메,두메 도곡
309,309,3220000-101-1991-13023,19911226,영업/정상,95.72,135887,서울특별시 강남구 신사동 511-19,서울 강남구 도산대로1길 16,취향의공간,한식,...,상수도전용,신사동,모범식당지정,취향의공간,['182478179'],182478179,604735311,https://place.map.kakao.com/604735311,취향의공간,취향의공간 신사


In [55]:
df.loc[310:320]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
310,310,3220000-101-1991-13094,19910128,영업/정상,38.64,135909,서울특별시 강남구 역삼동 637-36번지,서울 강남구 강남대로94길 75,신대흥장,중국식,...,상수도전용,역삼동,NaN,신대흥장,['849573094'],849573094,849573094,https://place.map.kakao.com/849573094,신대흥장,신대흥장 역삼
311,311,3220000-101-1991-15165,19911010,영업/정상,91.20,135965,서울특별시 강남구 개포동 1260-8번지,서울 강남구 개포로31길 5,장군숯불갈비,한식,...,상수도전용,개포동,NaN,장군숯불갈비,['16040568'],16040568,16040568,https://place.map.kakao.com/16040568,장군숯불갈비,장군숯불갈비 개포
312,312,3220000-101-1991-15196,19910919,영업/정상,22.05,135994,서울특별시 강남구 개포동 186-17번지,서울 강남구 개포로 512,시나브로,한식,...,상수도전용,개포동,NaN,시나브로,NaN,,,,시나브로,시나브로 개포
313,313,3220000-101-1991-15435,19911217,영업/정상,113.03,135841,서울특별시 강남구 대치동 906-21번지,서울 강남구 역삼로 421,두꺼비,한식,...,상수도전용,대치동,NaN,두꺼비,['16734080'],16734080,16734080,https://place.map.kakao.com/16734080,두꺼비,두꺼비 대치
314,314,3220000-101-1991-15437,1991-05-24,영업/정상,264.46,135-501,서울특별시 강남구 대치동 994-8,서울 강남구 영동대로 221,장독대,한식,...,NaN,대치동,NaN,장독대,NaN,,,,장독대,장독대 대치
315,315,3220000-101-1991-15467,1991-05-02,영업/정상,41.60,135-994,서울특별시 강남구 개포동 186-12,서울 강남구 개포로82길 7,투다리,한식,...,NaN,개포동,NaN,투다리,NaN,,,,투다리,투다리 개포
316,316,3220000-101-1991-15484,19910201,영업/정상,77.40,135840,서울특별시 강남구 대치동 896-2번지,서울 강남구 테헤란로64길 27,한성양꼬치,한식,...,상수도전용,대치동,NaN,한성양꼬치,['658037929'],658037929,658037929,https://place.map.kakao.com/658037929,한성양꼬치,한성양꼬치 대치
317,317,3220000-101-1991-15731,1991-09-01,영업/정상,161.31,135-878,"서울특별시 강남구 삼성동 145-2 지상1,2층",서울 강남구 봉은사로 504,길목 신관,한식,...,NaN,삼성동,NaN,길목 신관,['17219874'],17219874,17219874,https://place.map.kakao.com/17219874,길목 신관,길목 신관 삼성
318,318,3220000-101-1991-15736,19910201,영업/정상,104.44,135877,서울특별시 강남구 삼성동 143-19번지,서울 강남구 테헤란로69길 10,전주순대,한식,...,상수도전용,삼성동,NaN,전주순대,['9108820'],9108820,9108820,https://place.map.kakao.com/9108820,전주순대,전주순대 삼성
319,319,3220000-101-1991-15740,19910301,영업/정상,26.78,135509,서울특별시 강남구 삼성동 119-0번지,서울 강남구 봉은사로 478,사또족발,한식,...,상수도전용,삼성동,NaN,사또족발,NaN,,,,사또족발,사또족발 삼성


In [53]:
df.loc[:5]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com/10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com/727661240,로비라운지,로비라운지 삼성
3,3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,...,상수도전용,삼성동,NaN,로비바,['21550576'],21550576,21550576,https://place.map.kakao.com/21550576,로비바,로비바 삼성
4,4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,...,상수도전용,삼성동,NaN,노블발렌티,"['17157328', '26597965']",17157328,17157328,https://place.map.kakao.com/17157328,노블발렌티,노블발렌티 삼성
5,5,3220000-101-1911-16480,19110528,영업/정상,90.75,135957,서울특별시 강남구 청담동 126-6번지,서울 강남구 압구정로79길 47,골목식당,한식,...,상수도전용,청담동,NaN,골목식당,NaN,,,,골목식당,골목식당 청담


In [28]:
df[df["keyword"]=="제주흑돼지마을"]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
8352,8352,3220000-101-2020-00608,2020-06-09,영업/정상,97.55,135-833,서울특별시 강남구 논현동 277-31 내외빌딩,서울 강남구 봉은사로 319,제주흑돼지마을,한식,...,NaN,논현동,NaN,제주흑돼지마을,['365061365'],365061365,365061365,https://place.map.kakao.com/365061365,제주흑돼지마을,제주흑돼지마을 논현


In [24]:
df.shape

(12528, 23)

In [21]:
len('''comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error
comments error'''.split("\n"))

297

## 여기까지

In [50]:
url

'https://place.map.kakao.com/main/v/ '

In [42]:
pd.DataFrame(comments)

""


In [51]:
reviews_df = pd.DataFrame(columns=("name", "user", 'review', 'date', 'rate', 
                                   "avg_rate", "review_cnt", "is_model"))
reviews_df

,name,user,review,date,rate,avg_rate,review_cnt,is_model


In [17]:
comments = []

for i, id in enumerate(df["id_mod2"][:1]):
    try:
        request_id = id
        url = f'https://place.map.kakao.com/main/v/{request_id}'
        response = requests.get(url)

        if response.status_code == 200:
            comment_data = json.loads(response.content)['comment']
            comment_count = int(comment_data['kamapComntcnt'])
            overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
            for c in comment_data['list']:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                request_comment_id = c['commentid']
            print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
        else:
            print('error')
        
        # try:
        while len(comments) < comment_count:
            url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
            response = requests.get(url)
            if response.status_code == 200:
                comment_data = json.loads(response.content)['comment']
                for c in comment_data['list']:
                    try:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': c['contents'], 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    except:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': "", 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    request_comment_id = c['commentid']
                reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
        #     else:
        #         # print('error in while')
        #         pass
        # except:
        #     pass        

    except:
        pass

error


In [52]:
# comments = []

# for i, id in enumerate(df["id_mod2"][:2]):
#     try:
#         request_id = id
#         url = f'https://place.map.kakao.com/main/v/{request_id}'
#         response = requests.get(url)

#         if response.status_code == 200:
#             comment_data = json.loads(response.content)['comment']
#             comment_count = int(comment_data['kamapComntcnt'])
#             overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
#             for c in comment_data['list']:
#                 comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
#                 request_comment_id = c['commentid']
#             print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         else:
#             print('error')
#         #     for c in comment_data['list']:
#         #         try:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': c['contents'], 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         except:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': "", 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         request_comment_id = c['commentid']
#         #     # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         #     # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#         # else:
#         #     # print('error')
#         #     pass



#         while len(comments) < comment_count:
#             url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
#             response = requests.get(url)
#             if response.status_code == 200:
#                 comment_data = json.loads(response.content)['comment']
#                 for c in comment_data['list']:
#                     try:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': c['contents'], 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     except:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': "", 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     request_comment_id = c['commentid']
#                 # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#             else:
#                 # print('error in while')
#                 pass


#     except:
#         pass

## selenium

In [57]:
df.head()

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com727661240,로비라운지,로비라운지 삼성
3,3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,...,상수도전용,삼성동,NaN,로비바,['21550576'],21550576,21550576,https://place.map.kakao.com21550576,로비바,로비바 삼성
4,4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,...,상수도전용,삼성동,NaN,노블발렌티,"['17157328', '26597965']",17157328,17157328,https://place.map.kakao.com17157328,노블발렌티,노블발렌티 삼성


In [ ]:
# 상세보기에 조회된 모범 음식점 목록을 조회

# store_name = "솥내음 스타필드 코엑스몰"
# page_url = "https://place.map.kakao.com/710342577"


store_names = df["name"]
page_urls = df["url"]
review_results = []

driver = wb.Chrome() # 브라우저 실행

for index, (store_name, page_url) in enumerate(zip(store_names, page_urls)):

    try:

        driver.get(page_url) # 음식점 상세 화면
        time.sleep(1) # 1초 쉬기


        # total_num = int((bs(driver.page_source,'lxml')).select("span.color_g")[0].text.split(")")[0][1:])
        
        soup = bs(driver.page_source,'lxml')
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수


        more_nums = (int(tot_num.text) - 3) // 5 + 1
        print(f"--------------- {index}. {store_name} 후기 더보기 ---------------")
        for more_num in range(more_nums):
            try:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a')
                btnMore.click()
                time.sleep(1)
            except:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
                btnMore.click()
                time.sleep(1)


        soup = bs(driver.page_source,'lxml')

        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
        users = contents_div.find_all(name="span", attrs={"class": "txt_username"})
        reviews_count = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[0::2]
        avg_rates = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[1::2]
        date = contents_div.find_all(name="span", attrs={"class": "time_write"})
        is_model = df.loc[index, "exemplary_diner"]

        print(f"--------------- {index}. {store_name} 리뷰 크롤링 ---------------")

        for idx in range(len(reviews)):

            # rate, review, user, review_count, avg_rate 지정
            user = users[idx].text
            rate = ((int(rates[idx].get("style")[6:-2]) / 100) * 5)    
            avg_rate = float(avg_rates[idx].text)
            review = reviews[idx].find(name="span").text
            review_count = reviews_count[idx].text
                
            # dict 파일 생성
            review_results.append({"store_name" : store_name,
                                "user" : user,                         
                                "rate" : rate,                                                    
                                "avg_rate" : avg_rate,
                                "review" : review,
                                "review_count" : review_count,
                                "date" : date,
                                "is_model" : is_model})
    
    except:
        print(f"카카오맵에 {store_name} 정보가 없음")



reviews_total_df = pd.DataFrame(review_results)

# url 채운 파일 csv로 저장
reviews_total_df.to_csv("data/reviews_total.csv", index=False, encoding="utf-8-sig")